캐글에서 다운받아 colab에 업로드시 너무 오래걸려 googleapis에서 다운받음

In [1]:
import os, shutil, zipfile
from keras import models, layers, optimizers

In [2]:
# If the URL doesn't work, visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL to the dataset

# Note: This is a very large dataset and will take time to download
!mkdir /tmp
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


mkdir: cannot create directory ‘/tmp’: File exists
--2020-10-04 08:46:59--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 173.222.113.209, 2600:1402:2000:193::e59, 2600:1402:2000:1bb::e59
Connecting to download.microsoft.com (download.microsoft.com)|173.222.113.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M   105MB/s    in 8.1s    

2020-10-04 08:47:08 (97.1 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



훈련, 검증, 테스트 폴더로 이미지 복사하기

In [3]:
original_dataset_dir ='/tmp/PetImages'  # 원본 데이터셋을 압축 해제한 디렉토리 경로
base_dir = '/tmp/cats_and_dogs_small' #소규모 데이타셋을 저장할 디렉토리

if not os.path.isdir(base_dir):
  os.mkdir(base_dir)
#end of if

#훈련, 검증, 테스트 분할을 위한 디렉토리
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

#훈련용 고양이 사진 디렉토리
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

#훈련용 강아지 사진 디렉토리
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

#검증용 고양이 사진 디렉토리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

#검증용 강아지 사진 디렉토리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

#테스트용 고양이 사진 디렉토리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

#테스트용 강아지 사진 디렉토리
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [4]:
#처음 1,000개의 고양이 이미지를 train_cats_dir에 복사한다.
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir+"/Cat", fname)
  dst = os.path.join(train_cats_dir, fname)
  shutil.copy(src, dst)
#end of for

#다음 500개의 고양이 이미지를 validation_cats_dir에 복사한다.
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir+"/Cat", fname)
  dst = os.path.join(validation_cats_dir, fname)
  shutil.copy(src, dst)
#end of fo

#다음음 500개의 고양이 이미지를 test_cats_dir에 복사한다.
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir+"/Cat", fname)
  dst = os.path.join(test_cats_dir, fname)
  shutil.copy(src, dst)
#end of for


#처음 1,000개의 강아지 이미지를 train_dogs_dir에 복사한다.
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir+"/Dog", fname)
  dst = os.path.join(train_dogs_dir, fname)
  shutil.copy(src, dst)
#end of for

#다음 500개의 강아지 이미지를 validation_dogs_dir에 복사한다.
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir+"/Dog", fname)
  dst = os.path.join(validation_dogs_dir, fname)
  shutil.copy(src, dst)
#end of for

#다음음 500개의 강아지 이미지를 test_dogs_dir에 복사한다.
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir+"/Dog", fname)
  dst = os.path.join(test_dogs_dir, fname)
  shutil.copy(src, dst)
#end of for

정상복사를 확인하기 위해 각 분할(훈련/검증/테스트)에 들어 있는 사진의 개수를 카운트

In [5]:
print("원본 고양이 이미지 전체 개수", len(os.listdir(original_dataset_dir+"/Cat")))
print("원본 강아지 이미지 전체 개수", len(os.listdir(original_dataset_dir+"/Dog")))
print("\n")
print("훈련용 고양이 이미지 전체 개수", len(os.listdir(train_cats_dir)))
print("훈련용 강아지 이미지 전체 개수", len(os.listdir(train_dogs_dir)))
print("\n")
print("검증용 고양이 이미지 전체 개수", len(os.listdir(validation_cats_dir)))
print("검증용 강아지 이미지 전체 개수", len(os.listdir(validation_dogs_dir)))
print("\n")
print("테스트용 고양이 이미지 전체 개수", len(os.listdir(test_cats_dir)))
print("테스트용 강아지 이미지 전체 개수", len(os.listdir(test_dogs_dir)))
print("\n")

원본 고양이 이미지 전체 개수 12501
원본 강아지 이미지 전체 개수 12501


훈련용 고양이 이미지 전체 개수 1000
훈련용 강아지 이미지 전체 개수 1000


검증용 고양이 이미지 전체 개수 500
검증용 강아지 이미지 전체 개수 500


테스트용 고양이 이미지 전체 개수 500
테스트용 강아지 이미지 전체 개수 500




고양이와 강아지를 분류를 위한 초규모 컨브넷 만들기

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

모델 컴파일

In [7]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

데이타 전처리: ImageDataGenerator를 사용하여 디렉토리에서 이미지 읽기

In [8]:
from keras.preprocessing.image import ImageDataGenerator

#모든 이미지를 1/255로 스케일을 조정한다.
train_datagen =  ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, #타깃 디렉토리
    target_size=(150,150), #모든 이미지를 150x150 크기로 바꿉니다.
    batch_size = 20, 
    class_mode = 'binary' )  #binary_crossentropy 손실을 사용하기 때문에 이진 레이블을 필요합니다.
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20,)


배치 제너레이터를 사용하여 모델 훈련하기

In [11]:
history =  model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator, 
    validation_steps=50)

Epoch 1/30
 94/100 [===========================>..] - ETA: 0s - loss: 0.6873 - acc: 0.5479

UnknownError: ignored

모델 저장하기

In [ ]:
model.save('cats_and_dogs_small_1.h5')